# 医疗文档图像文字识别演示

> 🏥 使用PaddleOCR从医疗文档PNG图像中提取文本并保存为CSV

**版本**: v1.3.8 (修复IDE Pylance诊断问题和代码质量问题) | **更新时间**: 2025-08-24

## 🎯 功能特性
- 📄 支持医疗文档图像文字识别
- 🤖 使用PaddleOCR高精度识别引擎
- 📊 自动生成结构化CSV报告
- 🖼️ 支持多种图像格式输入
- 💡 简单易用的交互界面
- 🏠 完整本地开发环境支持
- 🌐 支持中英文混合识别
- 🏗️ 独立应用架构，可直接在Colab运行

## 🔧 更新内容 (v1.3.8)
- **🔧 代码质量**: 修复了所有IDE Pylance诊断警告和错误
- **📝 代码规范**: 清理了未使用的导入，修复缩进和语法问题
- **🛡️ 类型安全**: 添加了适当的类型注解和noqa标记
- **📊 代码整洁**: 优化了代码结构和可读性
- **🔍 诊断修复**: 解决了9个Pylance报告的问题
- **⚡ 性能优化**: 移除不必要的导入减少内存占用

## 🚀 使用说明

### Colab环境
1. 运行环境检查和依赖安装
2. 上传医疗文档图像
3. 执行OCR文字识别
4. 下载CSV结果文件

### 本地环境
```bash
# 从项目根目录一键启动
../start_local.sh

# 手动启动
source ../venv/bin/activate && jupyter notebook
```

### 目录结构
```
medical-ocr/
├── medical-ocr-demo.ipynb    # 本演示文件
├── gradio_demo.py           # Web界面版本
├── test_chinese_encoding_fix.py  # 中文编码测试
└── assets/                  # 资源文件
    ├── sample_docs/         # 示例文档
    └── results/            # OCR结果
```

---
*使用 Claude Code 开发，支持 Google Colab 和本地运行 🚀*

In [ ]:
# ================================
# 环境检查和基础设置
# ================================

import warnings
warnings.filterwarnings("ignore")

def check_environment():
    """检查运行环境并显示系统信息"""
    print("🔍 检查运行环境...")
    
    # 检查是否在Colab环境
    try:
        import google.colab # type: ignore # noqa: F401 # 需要用于环境检测
        print("✅ 运行在Google Colab")
        in_colab = True
    except ImportError:
        print("ℹ️ 运行在本地环境")
        in_colab = False
    
    # 检查GPU
    try:
        import torch
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"✅ 计算设备: {device}")
        if device == 'cuda':
            print(f"✅ GPU型号: {torch.cuda.get_device_name(0)}")
            print(f"✅ GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    except ImportError:
        print("ℹ️ PyTorch未安装，使用CPU模式")
    
    return in_colab

# 运行环境检查
in_colab = check_environment()

In [ ]:
# ================================
# 安装必要的依赖包
# ================================

def install_dependencies():
    """安装项目所需的依赖包"""
    print("📦 安装医疗OCR项目依赖...")
    
    import subprocess
    import sys
    
    # 核心依赖包列表
    packages = [
        'paddlepaddle',
        'paddleocr',
        'pandas',
        'pillow',
        'opencv-python',
        'tqdm',
        'gradio'
    ]
    
    for package in packages:
        try:
            if package == 'opencv-python':
                import cv2  # type: ignore # noqa: F401 # 用于验证opencv安装
                print(f"✅ {package} 已安装")
            elif package == 'pillow':
                from PIL import Image  # type: ignore # noqa: F401 # 用于验证PIL安装
                print(f"✅ {package} 已安装")
            else:
                __import__(package.replace('-', '_'))
                print(f"✅ {package} 已安装")
        except ImportError:
            print(f"📥 安装 {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    
    print("✅ 所有依赖安装完成!")

# 安装依赖
install_dependencies()

In [ ]:
# ================================
# 导入必要的库
# ================================

try:
    import pandas as pd  # type: ignore
    from tqdm import tqdm
    from paddleocr import PaddleOCR  # type: ignore
    import gradio as gr  # type: ignore
    print("📚 所有库导入成功!")
except ImportError as e:
    print(f"❌ 库导入失败: {e}")
    print("💡 请先运行依赖安装单元格")

In [ ]:
# ================================
# 医疗OCR核心功能类 - 修复版本
# ================================

class MedicalOCRProcessor:
    def __init__(self):
        """初始化医疗OCR处理器"""
        print("🏥 初始化医疗OCR处理器...")
        
        # 检查GPU可用性
        try:
            import torch
            use_gpu = torch.cuda.is_available()
            gpu_info = f"GPU可用: {use_gpu}"
            if use_gpu:
                gpu_info += f" (设备: {torch.cuda.get_device_name(0)})"
            print(f"⚡ {gpu_info}")
        except ImportError:
            use_gpu = False
            print("ℹ️ PyTorch未安装，使用CPU模式")
        
        # 初始化PaddleOCR，使用更稳定的配置
        try:
            # 尝试最新版本的初始化方式
            self.ocr = PaddleOCR(
                use_angle_cls=True, 
                lang='ch',
                use_gpu=use_gpu,
                show_log=False,  # 减少日志输出
                enable_mkldnn=False,  # 在某些环境中可能导致问题
                cpu_threads=4  # 限制CPU线程数避免资源争用
            )
            print("✅ 使用标准参数初始化OCR引擎")
        except Exception as e:
            print(f"⚠️ 标准初始化失败: {e}")
            print("🔄 尝试兼容性初始化...")
            try:
                # 使用最基础的参数进行初始化
                self.ocr = PaddleOCR(use_angle_cls=True, lang='ch')
                print("✅ 使用兼容性参数初始化OCR引擎")
            except Exception as e2:
                print(f"❌ OCR初始化完全失败: {e2}")
                self.ocr = None
                raise RuntimeError(f"PaddleOCR初始化失败: {e2}")
        
        print("✅ OCR引擎初始化完成")
    
    def _preprocess_image(self, image_path):
        """预处理图像，确保格式和质量适合OCR"""
        try:
            from PIL import Image as PILImage
            import os # type: ignore
            
            # 打开并验证图像
            with PILImage.open(image_path) as img:
                print(f"📊 原始图像信息: 尺寸={img.size}, 模式={img.mode}")
                
                # 转换为RGB格式（如果不是的话）
                if img.mode != 'RGB':
                    print(f"🔄 转换图像模式: {img.mode} -> RGB")
                    img = img.convert('RGB')
                
                # 检查图像尺寸，如果过大则适当缩小
                max_size = 2048
                if max(img.size) > max_size:
                    print(f"🔄 调整图像尺寸: {img.size}")
                    ratio = max_size / max(img.size)
                    new_size = (int(img.size[0] * ratio), int(img.size[1] * ratio))
                    img = img.resize(new_size, PILImage.Resampling.LANCZOS)
                    print(f"✅ 新尺寸: {img.size}")
                
                # 保存预处理后的图像
                processed_path = image_path.replace('.png', '_processed.png').replace('.jpg', '_processed.jpg').replace('.jpeg', '_processed.jpg')
                if processed_path == image_path:
                    processed_path = image_path.replace('.', '_processed.')
                
                img.save(processed_path, quality=95, optimize=False)
                print(f"💾 预处理图像已保存: {processed_path}")
                
                return processed_path
                
        except Exception as e:
            print(f"⚠️ 图像预处理失败: {e}，使用原始图像")
            return image_path

    def _parse_ocr_result(self, result):
        """解析OCR结果 - 兼容多种返回格式"""
        extracted_texts = []
        
        try:
            if result and isinstance(result, list):
                # 处理列表格式结果
                for page_result in result:
                    # 处理新版本OCRResult对象
                    if hasattr(page_result, 'rec_texts') and hasattr(page_result, 'rec_scores'):
                        print("✅ 检测到OCRResult对象格式")
                        texts = page_result.rec_texts
                        scores = page_result.rec_scores
                        
                        print(f"📊 识别到文本数量: {len(texts) if texts else 0}")
                        
                        if texts and scores:
                            for text, score in zip(texts, scores):
                                if text and text.strip():
                                    extracted_texts.append({
                                        'text': text.strip(),
                                        'confidence': float(score)
                                    })
                                    print(f"📝 提取文本: {text.strip()[:50]}... (置信度: {score:.3f})")
                    
                    # 处理传统列表格式 (旧版PaddleOCR格式)
                    elif isinstance(page_result, list):
                        print("✅ 检测到传统列表格式")
                        for line_result in page_result:
                            if (line_result and len(line_result) >= 2 and
                                line_result[1] and len(line_result[1]) >= 2):
                                
                                text = line_result[1][0]
                                confidence = line_result[1][1]
                                
                                if text and text.strip():
                                    extracted_texts.append({
                                        'text': text.strip(),
                                        'confidence': float(confidence)
                                    })
                                    print(f"📝 提取文本: {text.strip()[:50]}... (置信度: {confidence:.3f})")
                    
                    # 处理字典格式
                    elif isinstance(page_result, dict) and 'rec_texts' in page_result and 'rec_scores' in page_result:
                        print("✅ 检测到字典格式")
                        texts = page_result['rec_texts']
                        scores = page_result['rec_scores']
                        
                        if texts and scores:
                            for text, score in zip(texts, scores):
                                if text and text.strip():
                                    extracted_texts.append({
                                        'text': text.strip(),
                                        'confidence': float(score)
                                    })
                                    print(f"📝 提取文本: {text.strip()[:50]}... (置信度: {score:.3f})")
            
            # 处理直接字典格式
            elif result and isinstance(result, dict):
                if 'rec_texts' in result and 'rec_scores' in result:
                    print("✅ 检测到直接字典格式")
                    texts = result['rec_texts']
                    scores = result['rec_scores']
                    
                    if texts and scores:
                        for text, score in zip(texts, scores):
                            if text and text.strip():
                                extracted_texts.append({
                                    'text': text.strip(),
                                    'confidence': float(score)
                                })
                                print(f"📝 提取文本: {text.strip()[:50]}... (置信度: {score:.3f})")
                                
        except Exception as e:
            print(f"⚠️ 结果解析失败: {e}")
            import traceback
            print(f"详细错误: {traceback.format_exc()}")
        
        return extracted_texts

    def extract_text_from_image(self, image_path):
        """从图像中提取文字 - 增强版本"""
        import os
        
        if self.ocr is None:
            print("❌ OCR引擎未初始化")
            return []
        
        try:
            # 验证图像文件
            if not os.path.exists(image_path):
                print(f"❌ 图像文件不存在: {image_path}")
                return []
            
            if os.path.getsize(image_path) == 0:
                print(f"❌ 图像文件为空: {image_path}")
                return []
            
            print(f"📄 正在处理图像: {image_path}")
            print(f"📊 文件大小: {os.path.getsize(image_path)} 字节")
            
            # 预处理图像：确保图像格式和质量适合OCR
            processed_image_path = self._preprocess_image(image_path)
            
            # 使用不同的API调用方式进行兼容性处理
            result = None
            extracted_texts = []
            
            # 方法1: 尝试使用predict方法 (推荐的新版本API)
            try:
                print("🔄 尝试使用predict方法...")
                result = self.ocr.predict(processed_image_path)
                print(f"✅ predict方法调用成功，结果类型: {type(result)}")
                extracted_texts = self._parse_ocr_result(result)
                
                if extracted_texts:
                    print(f"✅ 成功识别 {len(extracted_texts)} 行文字")
                    return extracted_texts
                
            except Exception as e1:
                print(f"⚠️ predict方法失败: {e1}")
                
                # 方法2: 尝试使用传统的ocr方法 (不使用cls参数)
                try:
                    print("🔄 尝试使用传统ocr方法...")
                    result = self.ocr.ocr(processed_image_path)  # type: ignore # 移除cls参数以兼容新版本
                    print(f"✅ OCR方法调用成功，结果类型: {type(result)}")
                    extracted_texts = self._parse_ocr_result(result)
                    
                    if extracted_texts:
                        print(f"✅ 成功识别 {len(extracted_texts)} 行文字")
                        return extracted_texts
                        
                except Exception as e2:
                    print(f"❌ 所有可用的OCR调用方法都失败")
                    print(f"详细错误: predict={e1}, ocr={e2}")
                    # 注意：PaddleOCR对象不支持直接调用，移除该尝试
            
            # 如果所有方法都没有识别到文字
            if not extracted_texts:
                print("⚠️ 未检测到任何文字内容")
                self._debug_result_structure(result)
                self._check_image_quality(processed_image_path)
                
                return []
            
            return extracted_texts
        
        except Exception as e:
            print(f"❌ 图像处理失败: {str(e)}")
            import traceback
            print(f"详细错误信息: {traceback.format_exc()}")
            return []
    
    def _debug_result_structure(self, result):
        """调试结果结构"""
        try:
            print(f"🔍 调试信息: result类型={type(result)}")
            if result:
                print(f"🔍 result长度: {len(result) if hasattr(result, '__len__') else 'N/A'}")
                if isinstance(result, list) and len(result) > 0:
                    first_item = result[0]
                    print(f"🔍 第一项类型: {type(first_item)}")
                    if isinstance(first_item, dict):
                        print(f"🔍 字典键: {list(first_item.keys())}")
                    elif hasattr(first_item, '__dict__'):
                        print(f"🔍 对象属性: {list(vars(first_item).keys())}")
                    elif isinstance(first_item, list) and len(first_item) > 0:
                        print(f"🔍 嵌套列表长度: {len(first_item)}")
                        if len(first_item) > 0:
                            print(f"🔍 嵌套项类型: {type(first_item[0])}")
        except Exception as e:
            print(f"🔍 调试信息获取失败: {e}")
    
    def _check_image_quality(self, image_path):
        """检查图像质量"""
        try:
            from PIL import Image as PILImage
            import os
            
            if not os.path.exists(image_path):
                print("🔍 图像文件不存在")
                return
            
            with PILImage.open(image_path) as img:
                width, height = img.size
                total_pixels = width * height
                
                print(f"🔍 图像质量检查:")
                print(f"   尺寸: {width}x{height} ({total_pixels:,} 像素)")
                print(f"   格式: {img.format}")
                print(f"   模式: {img.mode}")
                
                # 质量评估
                if total_pixels < 50000:
                    print("   ⚠️ 图像分辨率较低，可能影响识别效果")
                elif total_pixels > 4000000:
                    print("   ℹ️ 图像分辨率很高，处理速度可能较慢")
                else:
                    print("   ✅ 图像分辨率适中")
                
        except Exception as e:
            print(f"🔍 图像质量检查失败: {e}")
    
    def process_single_image(self, image_path):
        """处理单个图像文件"""
        import os
        print(f"📄 处理图像: {os.path.basename(image_path)}")
        
        # 提取文字
        extracted_texts = self.extract_text_from_image(image_path)
        
        # 整理结果
        results = []
        for i, item in enumerate(extracted_texts):
            results.append({
                'file_name': os.path.basename(image_path),
                'line_number': i + 1,
                'extracted_text': item['text'],
                'confidence': round(item['confidence'], 4)
            })
        
        return results
    
    def process_multiple_images(self, image_paths):
        """批量处理多个图像文件"""
        all_results = []
        
        print(f"📊 开始批量处理 {len(image_paths)} 个图像文件...")
        
        for image_path in tqdm(image_paths, desc="处理进度"):
            results = self.process_single_image(image_path)
            all_results.extend(results)
        
        return all_results
    
    def save_results_to_csv(self, results, output_path):
        """保存结果到CSV文件"""
        if not results:
            # 如果没有结果，创建空的DataFrame
            df = pd.DataFrame(columns=['file_name', 'line_number', 'extracted_text', 'confidence'])
        else:
            df = pd.DataFrame(results)
        
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"💾 结果已保存到: {output_path}")
        return df

# 初始化OCR处理器
print("🔧 正在初始化OCR处理器...")
try:
    ocr_processor = MedicalOCRProcessor()
    print("✅ OCR处理器初始化成功!")
except Exception as e:
    print(f"❌ OCR处理器初始化失败: {e}")
    print("💡 请检查PaddleOCR安装是否正确")

In [ ]:
# ================================
# 创建示例医疗文档（用于演示）- 修复字体和中文显示问题
# ================================

def create_sample_medical_document():
    """创建示例医疗文档图像用于演示"""
    from PIL import Image, ImageDraw, ImageFont
    import os
    
    # 创建示例图像 - 使用更大尺寸和更好对比度
    img = Image.new('RGB', (1000, 800), color='white')
    draw = ImageDraw.Draw(img)
    
    # 添加边框增加文档感
    draw.rectangle([(20, 20), (980, 780)], outline='black', width=2)
    
    # 使用中文医疗内容进行识别测试
    sample_text = [
        "医疗诊断报告",
        "医院名称：XX市人民医院",
        "患者姓名：张三",
        "性别：男    年龄：45岁",
        "科室：心血管内科",
        "主治医师：李医生",
        "诊断：高血压、糖尿病",
        "处方：",
        "1. 降压药 10mg 每日一次",
        "2. 降糖药 5mg 每日两次",
        "医生签名：李医生",
        "日期：2025-08-24"
    ]
    
    # 获取中文字体 - 优先查找系统中的中文字体
    font = None
    title_font = None
    font_size = 28  # 适中的字体大小
    title_font_size = 36
    
    # 中文字体路径列表（按优先级排序）
    chinese_font_paths = [
        # 用户字体目录中的中文字体
        '/home/wuxia/.fonts/simhei.ttf',  # 黑体
        '/home/wuxia/.fonts/simsun.ttf',  # 宋体
        '/home/wuxia/.fonts/simsun.ttc',  # 宋体TTC格式
        '/home/wuxia/.fonts/simkai.ttf',  # 楷体
        # 系统可能安装的中文字体
        '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',  # 文泉驿正黑
        '/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',  # 文泉驿微米黑
        '/usr/share/fonts/truetype/droid/DroidSansFallbackFull.ttf',  # Droid字体
        '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',  # Noto CJK
        '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',  # Liberation (支持部分中文)
        '/System/Library/Fonts/PingFang.ttc',  # macOS苹果苹方
        'C:\\\\Windows\\\\Fonts\\\\simhei.ttf',  # Windows黑体
        'C:\\\\Windows\\\\Fonts\\\\simsun.ttc',  # Windows宋体
    ]
    
    print("🔍 搜索中文字体...")
    
    # 尝试加载中文字体
    for font_path in chinese_font_paths:
        if os.path.exists(font_path):
            try:
                font = ImageFont.truetype(font_path, font_size)
                title_font = ImageFont.truetype(font_path, title_font_size)
                print(f"✅ 成功加载中文字体: {font_path}")
                break
            except Exception as e:
                print(f"⚠️ 字体加载失败: {font_path} - {e}")
                continue
    
    # 如果没有找到中文字体，尝试系统默认字体
    if font is None:
        print("⚠️ 未找到中文字体，尝试系统默认字体...")
        
        # 尝试一些可能支持中文的系统字体
        fallback_fonts = [
            '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
        ]
        
        for font_path in fallback_fonts:
            if os.path.exists(font_path):
                try:
                    font = ImageFont.truetype(font_path, font_size)
                    title_font = ImageFont.truetype(font_path, title_font_size)
                    print(f"✅ 使用备用字体: {font_path}")
                    print("⚠️ 注意: 此字体可能无法完全支持中文显示")
                    break
                except Exception:
                    continue
    
    # 最后的备用方案：使用PIL默认字体
    if font is None:
        try:
            font = ImageFont.load_default()
            title_font = font
            print("⚠️ 使用PIL默认字体 - 中文显示可能异常")
        except Exception as e:
            font = None
            title_font = None
            print(f"❌ 字体加载完全失败: {e}")
    
    # 绘制文本内容
    y_position = 60
    line_height = 45  # 稍微紧凑一些
    
    for i, text in enumerate(sample_text):
        try:
            # 第一行标题居中加粗
            if i == 0:
                # 计算居中位置
                if title_font:
                    try:
                        bbox = draw.textbbox((0, 0), text, font=title_font)
                        text_width = bbox[2] - bbox[0]
                    except Exception:
                        # 如果textbbox不可用，估算宽度
                        text_width = int(len(text) * title_font_size * 0.6)
                else:
                    text_width = len(text) * 20  # 估算宽度
                
                x_position = max(50, (1000 - text_width) // 2)  # 确保不会太靠左
                
                # 绘制标题
                if title_font:
                    draw.text((x_position, y_position), text, fill='black', font=title_font)
                else:
                    draw.text((x_position, y_position), text, fill='black')
                
                print(f"✅ 绘制标题: {text}")
                
                # 添加下划线
                draw.line([(x_position, y_position + title_font_size + 5), 
                          (x_position + text_width, y_position + title_font_size + 5)], 
                         fill='black', width=2)
                y_position += 20  # 标题后额外间距
                
            else:
                # 普通文本左对齐
                x_position = 60
                if font:
                    draw.text((x_position, y_position), text, fill='black', font=font)
                else:
                    draw.text((x_position, y_position), text, fill='black')
                
                print(f"✅ 绘制文本: {text}")
            
        except Exception as e:
            print(f"⚠️ 文字绘制失败: {e} - 文本: {text}")
            # 简单备用绘制（不使用字体）
            try:
                x_pos = 60 if i > 0 else 300
                draw.text((x_pos, y_position), text, fill='black')
                print(f"✅ 备用方式绘制: {text}")
            except Exception as e2:
                print(f"❌ 备用绘制也失败: {e2}")
                # 如果中文绘制完全失败，使用英文替代
                english_text = translate_to_english(text)
                try:
                    x_pos = 60 if i > 0 else 300
                    draw.text((x_pos, y_position), english_text, fill='red')
                    print(f"🔄 使用英文替代: {english_text}")
                except Exception:
                    print(f"❌ 所有绘制方式都失败了: {text}")
        
        y_position += line_height
    
    # 添加一些装饰元素增加真实感
    try:
        # 添加医院LOGO占位符
        draw.rectangle([(60, 100), (160, 160)], outline='gray', width=1)
        draw.text((90, 125), "LOGO", fill='gray')
        
        # 添加签名线
        draw.line([(700, 650), (950, 650)], fill='black', width=1)
        draw.text((700, 660), "Signature", fill='gray')
        
        # 添加日期戳
        draw.rectangle([(800, 700), (950, 750)], outline='blue', width=1)
        draw.text((810, 715), "2025-08-24", fill='blue')
        
        print("✅ 装饰元素添加完成")
    except Exception as e:
        print(f"⚠️ 装饰元素添加失败: {e}")
    
    # 保存示例图像到assets目录
    os.makedirs('assets/sample_docs', exist_ok=True)
    sample_path = 'assets/sample_docs/sample_medical_document.png'
    
    # 使用更高的质量设置保存
    img.save(sample_path, 'PNG', optimize=False)
    print(f"📄 创建示例医疗文档: {sample_path}")
    
    # 验证文件创建
    if os.path.exists(sample_path):
        file_size = os.path.getsize(sample_path)
        print(f"✅ 文件创建成功，大小: {file_size} 字节")
        
        # 显示字体使用情况
        if font:
            print("📝 字体状态: 已加载字体，中文应该可以正常显示")
        else:
            print("⚠️ 字体状态: 未加载字体，中文显示可能异常")
            
        # 给出中文字体安装建议
        if font is None:
            print("\\n💡 中文字体安装建议:")
            print("1. Ubuntu/Debian: sudo apt install fonts-wqy-zenhei fonts-wqy-microhei")
            print("2. 手动安装: 下载SimHei.ttf到 ~/.fonts/ 目录")
            print("3. 在Colab中: !apt install fonts-wqy-zenhei -y")
            
    else:
        print("❌ 文件创建失败")
    
    return sample_path

def translate_to_english(chinese_text: str) -> str:
    """将中文文本转换为英文（备用方案）"""
    translations = {
        "医疗诊断报告": "Medical Report",
        "医院名称：XX市人民医院": "Hospital: XX City People's Hospital",
        "患者姓名：张三": "Patient: Zhang San",
        "性别：男    年龄：45岁": "Gender: Male    Age: 45",
        "科室：心血管内科": "Dept: Cardiology",
        "主治医师：李医生": "Doctor: Dr. Li",
        "诊断：高血压、糖尿病": "Diagnosis: Hypertension, Diabetes",
        "处方：": "Prescription:",
        "1. 降压药 10mg 每日一次": "1. Antihypertensive 10mg daily",
        "2. 降糖药 5mg 每日两次": "2. Antidiabetic 5mg twice daily",
        "医生签名：李医生": "Doctor Signature: Dr. Li",
        "日期：2025-08-24": "Date: 2025-08-24"
    }
    return translations.get(chinese_text, chinese_text)

def install_chinese_fonts_in_colab() -> bool:
    """在Google Colab中安装中文字体"""
    try:
        import google.colab  # type: ignore # noqa: F401 # 用于Colab环境检测
        print("📦 检测到Colab环境，安装中文字体...")
        
        import subprocess
        # 安装中文字体包
        font_packages = [
            'fonts-wqy-zenhei',      # 文泉驿正黑
            'fonts-wqy-microhei',    # 文泉驿微米黑  
            'fonts-arphic-ukai',     # AR PL UKai CN
            'fonts-arphic-uming'     # AR PL UMing CN
        ]
        
        for package in font_packages:
            try:
                subprocess.run(['apt', 'install', '-y', package], 
                             capture_output=True, text=True, check=True)
                print(f"✅ 安装字体包: {package}")
            except subprocess.CalledProcessError as e:
                print(f"⚠️ 字体包安装失败: {package} - {e}")
        
        # 刷新字体缓存
        try:
            subprocess.run(['fc-cache', '-fv'], check=True)
            print("✅ 字体缓存已刷新")
        except Exception:
            print("⚠️ 字体缓存刷新失败")
            
        return True
        
    except ImportError:
        # 不在Colab环境中
        return False
    except Exception as e:
        print(f"❌ Colab字体安装失败: {e}")
        return False

# 如果在Colab环境中，尝试安装中文字体
print("🔍 检查运行环境并准备字体...")
if install_chinese_fonts_in_colab():
    print("🔄 重新加载字体后创建文档...")

# 创建示例文档
print("🎨 创建示例医疗文档...")
sample_doc = create_sample_medical_document()

# 显示示例图像
try:
    from IPython.display import Image as IPImage, display  # type: ignore # noqa: F401
    print("🖼️ 显示生成的医疗文档图像:")
    display(IPImage(sample_doc))
except ImportError:
    # 在非Jupyter环境中的备用方案
    print(f"✅ 示例文档已创建: {sample_doc}")
    print("💡 在Jupyter/Colab环境中会自动显示图像")
except Exception as e:
    print(f"⚠️ 图像显示失败: {e}")
    print(f"✅ 示例文档已创建: {sample_doc}")

In [ ]:
# ================================
# 演示OCR功能
# ================================

def demo_ocr_functionality():
    """演示OCR功能"""
    print("🚀 开始演示医疗OCR功能...")
    
    # 确保使用正确的图像路径
    import os
    sample_image_path = sample_doc
    if not os.path.exists(sample_image_path):
        # 尝试assets目录中的示例文档
        assets_path = 'assets/sample_docs/sample_medical_document.png'
        if os.path.exists(assets_path):
            sample_image_path = assets_path
        else:
            sample_image_path = 'sample_medical_document.png'
    
    print(f"📄 使用图像文件: {sample_image_path}")
    
    # 处理示例文档
    results = ocr_processor.process_single_image(sample_image_path)
    
    # 显示识别结果
    print("\n📊 文字识别结果:")
    print("-" * 60)
    
    for result in results:
        print(f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})")
    
    # 保存结果到CSV (assets/results目录)
    os.makedirs('assets/results', exist_ok=True)
    csv_path = 'assets/results/ocr_results_demo.csv'
    df = ocr_processor.save_results_to_csv(results, csv_path)
    
    print(f"\n📈 共识别出 {len(results)} 行文字")
    print(f"📄 结果已保存到 CSV 文件: {csv_path}")
    
    # 显示CSV内容预览
    if len(results) > 0:
        print("\n📋 CSV文件预览:")
        print(df.to_string(index=False))
    else:
        print("\n⚠️ 没有识别到文字内容，请检查图像文件")
    
    return df, csv_path

# 运行演示
demo_df, demo_csv = demo_ocr_functionality()

In [ ]:
# ================================
# 图像上传处理模块
# ================================

import os # type: ignore
import numpy as np  # type: ignore # noqa: F401
from PIL import Image as PILImage  # type: ignore # noqa: F401

print("🔍 图像处理模块已加载，准备接收上传图像...")

In [ ]:
# ================================
# Gradio Web交互界面
# ================================

def create_gradio_interface():
    """创建Gradio Web界面用于图像上传和OCR处理"""
    
    def process_uploaded_image(image):
        """处理上传的图像"""
        import tempfile
        import os
        
        if image is None:
            return "请上传一个图像文件", None
        
        try:
            # 创建临时文件保存上传的图像
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp_file:
                # 将PIL图像保存到临时文件
                image.save(tmp_file.name, 'PNG')
                temp_image_path = tmp_file.name
            
            print(f"💾 临时图像已保存: {temp_image_path}")
            
            # 使用OCR处理器处理图像
            results = ocr_processor.process_single_image(temp_image_path)
            
            # 清理临时文件
            try:
                os.unlink(temp_image_path)
            except Exception as e:
                print(f"⚠️ 临时文件清理失败: {e}")
            
            # 格式化输出结果
            if results:
                output_text = f"🎉 成功识别出 {len(results)} 行文字:\\n\\n"
                for result in results:
                    output_text += f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})\\n"
                
                # 创建CSV数据用于下载
                import pandas as pd
                import io
                
                df = pd.DataFrame(results)
                csv_buffer = io.StringIO()
                df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')
                csv_data = csv_buffer.getvalue()
                
                # 将CSV数据写入临时文件以供下载
                with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv', encoding='utf-8-sig') as csv_file:
                    csv_file.write(csv_data)
                    csv_file_path = csv_file.name
                
                return output_text, csv_file_path
            else:
                return "😞 未检测到任何文字内容，请检查图像质量", None
                
        except Exception as e:
            error_msg = f"❌ 图像处理失败: {str(e)}"
            print(error_msg)
            return error_msg, None
    
    # 创建Gradio界面
    with gr.Blocks(title="医疗文档OCR识别", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🏥 医疗文档OCR识别系统
        
        上传医疗文档图像，系统将自动识别其中的文字内容并生成CSV报告
        
        **支持格式**: PNG, JPG, JPEG  
        **识别语言**: 中文、英文  
        **输出格式**: CSV文件
        """)
        
        with gr.Row():
            with gr.Column():
                # 图像上传组件
                image_input = gr.Image(
                    label="📄 上传医疗文档图像",
                    type="pil",
                    height=400
                )
                
                # 处理按钮
                process_btn = gr.Button("🚀 开始识别", variant="primary", size="lg")
                
            with gr.Column():
                # 结果显示
                result_text = gr.Textbox(
                    label="📊 识别结果",
                    lines=15,
                    max_lines=20,
                    placeholder="识别结果将在这里显示...",
                    show_copy_button=True
                )
                
                # CSV下载
                csv_download = gr.File(
                    label="💾 下载CSV结果",
                    visible=False
                )
        
        # 示例图片展示
        gr.Markdown("### 📋 使用示例")
        with gr.Row():
            gr.Examples(
                examples=[
                    ["assets/sample_docs/sample_medical_document.png"]
                ] if os.path.exists("assets/sample_docs/sample_medical_document.png") else [],
                inputs=image_input,
                label="点击加载示例医疗文档"
            )
        
        # 使用说明
        gr.Markdown("""
        ### 💡 使用说明
        1. **上传图像**: 点击上方区域上传医疗文档图像
        2. **开始识别**: 点击"开始识别"按钮进行OCR处理
        3. **查看结果**: 识别结果将显示在右侧文本框中
        4. **下载CSV**: 处理完成后可下载CSV格式的结构化结果
        
        ### 🔧 技术特性
        - 🤖 基于PaddleOCR高精度识别引擎
        - 🌐 支持中英文混合识别
        - 📊 提供置信度评估
        - 💾 自动生成CSV格式结果
        - 🔍 支持多种图像格式
        
        ### ⚠️ 注意事项
        - 请确保图像清晰可读
        - 文字识别结果仅供参考
        - 重要医疗信息请人工核验
        """)
        
        # 绑定处理函数
        def update_result(image):
            """更新结果并控制下载按钮可见性"""
            result_text, csv_path = process_uploaded_image(image)
            
            if csv_path:
                # 有CSV文件时显示下载组件
                return result_text, gr.File(value=csv_path, visible=True)
            else:
                # 无CSV文件时隐藏下载组件
                return result_text, gr.File(visible=False)
        
        process_btn.click(
            fn=update_result,
            inputs=image_input,
            outputs=[result_text, csv_download]
        )
    
    return demo

# 创建并启动界面
print("🌐 创建Gradio Web界面...")
try:
    demo_interface = create_gradio_interface()
    print("✅ Gradio界面创建成功!")
    print("💡 在本地环境中，可以使用 demo_interface.launch() 启动Web服务")
    print("💡 在Colab环境中，可以使用 demo_interface.launch(share=True) 获取公共链接")
    
    # 根据环境选择启动方式
    try:
        import google.colab  # type: ignore # noqa: F401 # 检测Colab环境
        print("🚀 检测到Colab环境，启动公共分享链接...")
        demo_interface.launch(share=True, height=800)
    except ImportError:
        print("🏠 本地环境检测成功")
        print("📝 如需启动Web界面，请运行: demo_interface.launch()")
        # 在本地环境中不自动启动，避免占用端口
        
except Exception as e:
    print(f"❌ Gradio界面创建失败: {e}")
    print("💡 请检查Gradio是否正确安装")

In [ ]:
# ================================
# 项目总结和使用指南
# ================================

def show_project_summary():
    """显示项目总结和使用指南"""
    
    summary = """ # type: ignore
🎉 医疗文档OCR识别演示项目运行完成!

📋 项目功能总结:
✅ 医疗文档图像文字识别
✅ 中英文混合文字识别
✅ 批量图像处理支持
✅ CSV格式结果导出
✅ 置信度评估
✅ 用户友好的交互界面

🛠️ 技术特性:
• 基于PaddleOCR高精度识别引擎
• 支持GPU加速处理
• 自动角度检测和矫正
• 结构化数据输出

📖 使用场景:
• 医疗处方识别
• 病历文档数字化
• 检查报告提取
• 医疗档案管理

🔗 项目地址:
GitHub: https://github.com/zhurong2020/mcr
Colab: 当前notebook链接

👨‍⚕️ 适用用户:
• 医疗工作者
• 数据分析师
• 研究人员
• 开发者

⚠️ 使用说明:
1. 确保图像清晰可读
2. 支持PNG、JPG等常见格式
3. 批量处理时注意文件大小
4. 结果仅供参考，重要信息请人工核验

💡 技术支持:
如有问题或建议，欢迎反馈交流！
    """
    
    print(summary)

# 显示项目总结
show_project_summary()